<a href="https://colab.research.google.com/github/Kelly-Lima25/TCC/blob/main/TCC_Utilizando_os_conceitos_de_PLN_para_extrair_palavras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usando a biblioteca ruppell
A biblioteca permite a extração de texto de arquivos pdf e converte para dataframe.


In [ ]:
!pip install ruppell

In [ ]:
import ruppell

ruppell.setup_language(language='por') #Configurando o idioma para a linguagem do arquivo pdf

#Montando o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Criando o dataframe a partir dos arquivos pdf.

In [ ]:
data_Brasil = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/Brasil')

data_China = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/China')

data_EUA = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/EUA')

data_Egito = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/Egito')

data_ReinoUnido = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/Reino Unido')

data_Ruanda = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/Ruanda')

data_Singapura = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/Singapura')

data_UE = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/União Europeia')

data_Vietna = ruppell.folder_to_dataframe(folder_path='/content/drive/MyDrive/TCC/País/Vietnã')


# Tratamento das palavras

Utilizando a biblioteca NLTK para fazer a tokenização das palavras

In [ ]:
import nltk

nltk.download("all")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
from nltk import tokenize


**1º tratamento:**  Tokenizando todas as palavras do texto, retirando palavras irrelevantes(ex: de, na, que, em...) e deixando o texto em minúsculo.

In [ ]:

def token_min(texto):
  palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
  frase_processada = list()

  token_espaco = tokenize.WhitespaceTokenizer() #Separa cada palavra do texto com espaço

  for doc in texto["text_raw"]:
      nova_frase = list()
      doc = doc.lower()
      palavras_texto = token_espaco.tokenize(doc)
      for palavra in palavras_texto:
          if palavra not in palavras_irrelevantes:
              nova_frase.append(palavra)
      frase_processada.append(' '.join(nova_frase))

  texto["tratamento_1"] = frase_processada

Importando a biblioteca re e definindo a função remover para substituir a regex dos tratamentos 2, 3 e 4

In [ ]:
import re

In [ ]:
def remover(textos, regex):
    if type(textos) == str:
        return regex.sub("", textos)
    else:
        return [regex.sub("", texto) for texto in textos] # Para conjunto de texto

**2º tratamento:** Remoção de pontuação

In [ ]:
regex_pontuacao = r"[^\w\s]"
regex_pontuacao_epecial = r'[º]+'

regex_pontu = re.compile("|".join([regex_pontuacao, regex_pontuacao_epecial]))

def remover_pontuacao(texto, coluna_texto):
  remocao_pontuacao = remover(coluna_texto, regex_pontu)
  texto["tratamento_2"] = remocao_pontuacao


3º Tratamento: Removendo dígitos e numeração romana

In [ ]:
def remover_numero(texto):

  regex_romano = r'\b(xc|xl|l?x?x?x?)(ix|iv|v?i{0,3})+\b'
  regex_numeral = r'(\d+)'

  regex_digito = re.compile("|".join([regex_romano, regex_numeral]))

  texto_sem_numeracao = remover(texto.tratamento_2, regex_digito)

  texto["tratamento_3"] = texto_sem_numeracao



4º Tratamento: Tratamento para a remoção de palavras irrelevantes semânticamente para o entendimento do contexto dos documentos.

In [ ]:
regex_China = re.compile(r"\b(organizações|lei|artigos|medidas|devem|acordo|china|república|relevantes|popular|disposições|administrativos|leis|deve|outras|outros|instituições|direitos|realizar|capítulo|tomar|melhorar|uso|sobre|artigo|fornecer|gerar|etc|termos|competentes|competente|relevante|ministério)\b")

regex_Brasil = re.compile(r"\b(sistemas|sistema|art|lei|competente|uso|desta|bem|utilização|medidas|avaliação|direito|sobre|funcionamento|adoção|forma|parágrafo|único|incluindo|riscos|meio|termos|previstos|práticas|caso|n|seguintes|pessoa|alto|caput)\b")

regex_EUA = re.compile(r"\b(desenvolver|nacional|secretário|dia|diretor|prazo|agência|data|ordem|medida|programa|autoridade|prática|capacidade|forma|presidente|modelo|termo|avaliação|resultado|atividade|setor|recurso|agente|parte|poder|implementação|base|estrutura|consulta|acesso|acordo|conselho|objetivo|fim|entidade|grupo|parágrafo|despacho|contexto|artigo|pedido|seg|conta|seção|padrõe|energia|orientaçõe|assunto|conjunto|maneira|caso|processo|incluindo|dias|sobre|apropriado|partir|conforme|estados|outras|lei|unidos|deverá|desta|a|b|c|d|e|f|g|h|j|k|m|deste|modelos|considerar|relacionadas|interna|relevantes|coordenação|orientações|outros|bem|devem|meio|qualquer|consistente|uso|federal|através)\b")

regex_Egito = re.compile(r"\b(número|d b d n h s|idf fdnl r t fdd dsf cc s g h t s|c e n c c m p u t d r s s t m s n f r m c a n f r m c a t c n g c n t f c n f r m a t c p r c a p d s s s q u s c b a e s n c c s s|s s t m s c m p u t d r s m u t m d p r g r m s n g n h r|c u t r|estratégia|egipto|pode|egito|uso|podem|nível|países|sobre|fase|diferentes|através|geral|tempo|numero|total|é|menos|principais|bem|todos|outros|setor|internacional|país|especialmente|escala|ponto|melhorar|relacionadas|classificação|nova|novo|outra|todo|onde|toda|muitos|desde|necessário|muita|maior|alguns|piloto|ajudar|cada|disso|ano|sendo|qualquer|etc|deve|sendo|devem|sector|longo|parte|tanto|ter|toda|máquina|além|abaixo|investigação|exemplo|incluindo|forma|portanto)\b")

regex_Singapura = re.compile(r"\b(modelo|podem|pode|decisão|modelos|exemplo|sobre|tomada|garantir|medidas|conjunto|devem|praticas|por|a|b|c|d|e|f|processo|considerar|algoritmos|partes|princípios|relevantes|sistemas|diferentes|interessadas|conjunto|organização|estrutura|estruturas|decisões|eou|risco|usar|incluindo|uso|organizações)\b")

regex_UE = re.compile(r"\b(artigo|elevado|conformidade|requisito|medida|comissão|capítulo|ato|avaliação|caso|procedimento|atividade|colocação|título|termo|utilização|finalidade|efeito|termos|parte|base|controlo|registo|ponto|estadosmembros|acordo|norma|notificação|técnica|organismo|resultado|c|execução|tido|prazo|elemento|sistemas|circunstância|coima|âmbito|fim|conjunto|aquando|tr|cópia|tempo|parágrafo|jornal|aspeto|pai|documentação|devem|deve|presente|autoridade|regulamento|n|requisitos|organismos|união|estabelecidos|ue|refere|medidas|anexo|notificados|competentes|atividades|nacional|qualquer|atos|incluindo|causa|efeitos|prevista|a|b|d|e|f|g|h|j|k|l|m|n|fornecedores|gestão|sobre|pode|todas|relação|pedido|disso|conta|utilizador|diretiva|necessárias|podem|quaisquer|seguintes|adequadas|referidos|outras|alínea|cada|nomeadamente|bem|desses|desse|ter|outros|sob|nacionais|inteligência|artificial)\b")

regex_Vietna = re.compile(r"\b(medidas|aplicações|sobre|alto|alta|partilha|centro|data|ministério|nacionais|vietname|implementação|melhorar|nacional)\b")

regex_Ruanda = re.compile(r"\b(ano|fiscal|nacional|sobre|estabelecer|realizar|número|resultado)\b")

regex_ReinoUnido = re.compile(r"\b(sobre|cid|reino|unido|forma|podem|regulamentação|pode|exemplo|incluindo|regulamentares|modelos|existentes|todo|funções|função|garantir)\b")

5º Tratamento: Removendo os espaços em branco

In [ ]:

regex_espaco = re.compile(r" +")

def substituir_por_espaco(textos, regex):
    if type(textos) == str:
        return regex.sub(" ", textos)
    else:
        return [regex.sub(" ", texto) for texto in textos]

def remover_espaco(texto):
  remocao_de_espaco = substituir_por_espaco(texto.tratamento_4, regex_espaco)

  texto["tratamento_5"] = remocao_de_espaco

##Aplicação dos tratamentos nos datraframes de cada país

1º Tratamento:

In [ ]:
token_min(data_Brasil)
token_min(data_China)
token_min(data_EUA)
token_min(data_Egito)
token_min(data_Singapura)
token_min(data_UE)
token_min(data_Vietna)
token_min(data_Ruanda)
token_min(data_ReinoUnido)

2º Tratamento:

In [ ]:
remover_pontuacao(data_Brasil,data_Brasil.tratamento_1)
remover_pontuacao(data_China, data_China.tratamento_1)
remover_pontuacao(data_EUA, data_EUA.tratamento_1)
remover_pontuacao(data_Egito, data_Egito.tratamento_1)
remover_pontuacao(data_Singapura, data_Singapura.tratamento_1)
remover_pontuacao(data_UE, data_UE.tratamento_1)
remover_pontuacao(data_Vietna, data_Vietna.tratamento_1)
remover_pontuacao(data_Ruanda, data_Ruanda.tratamento_1)
remover_pontuacao(data_ReinoUnido, data_ReinoUnido.tratamento_1)

3º Tratamento:

In [ ]:
remover_numero(data_Brasil)
remover_numero(data_China)
remover_numero(data_EUA)
remover_numero(data_Egito)
remover_numero(data_Singapura)
remover_numero(data_UE)
remover_numero(data_Vietna)
remover_numero(data_Ruanda)
remover_numero(data_ReinoUnido)

4º Tratamento:

In [ ]:
tratamento_4_Brasil = remover(data_Brasil.tratamento_3, regex_Brasil)
data_Brasil["tratamento_4"] = tratamento_4_Brasil

In [ ]:
tratamento_4_China = remover(data_China.tratamento_3, regex_China)
data_China["tratamento_4"] = tratamento_4_China

In [ ]:
tratamento_4_EUA = remover(data_EUA.tratamento_3, regex_EUA)
data_EUA["tratamento_4"] = tratamento_4_EUA

In [ ]:
tratamento_4_Egito = remover(data_Egito.tratamento_3, regex_Egito)
data_Egito["tratamento_4"] = tratamento_4_Egito

In [ ]:
tratamento_4_Singapura = remover(data_Singapura.tratamento_3, regex_Singapura)
data_Singapura["tratamento_4"] = tratamento_4_Singapura

In [ ]:
tratamento_4_UE = remover(data_UE.tratamento_3, regex_UE)
data_UE["tratamento_4"] = tratamento_4_UE

In [ ]:
tratamento_4_Vietna = remover(data_Vietna.tratamento_3, regex_Vietna)
data_Vietna["tratamento_4"] = tratamento_4_Vietna

In [ ]:
tratamento_4_Ruanda = remover(data_Ruanda.tratamento_3, regex_Ruanda)
data_Ruanda["tratamento_4"] = tratamento_4_Ruanda

In [ ]:
tratamento_4_ReinoUnido = remover(data_ReinoUnido.tratamento_3, regex_ReinoUnido)
data_ReinoUnido["tratamento_4"] = tratamento_4_ReinoUnido

 5º Tratamento:

In [ ]:
remover_espaco(data_Brasil)
remover_espaco(data_China)
remover_espaco(data_EUA)
remover_espaco(data_Egito)
remover_espaco(data_Singapura)
remover_espaco(data_UE)
remover_espaco(data_Vietna)
remover_espaco(data_Ruanda)
remover_espaco(data_ReinoUnido)

#Plotagem das tabelas com as 12 palavras mais citadas

Calculando a frequência com que cada palavra aparece em cada um dos documentos

In [ ]:
token_espaco = tokenize.WhitespaceTokenizer()
todas_palavras_Brasil = ' '.join([texto for texto in data_Brasil["tratamento_5"]])
frequencia_Brasil = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Brasil))

todas_palavras_China = ' '.join([texto for texto in data_China["tratamento_5"]])
frequencia_China = nltk.FreqDist(token_espaco.tokenize(todas_palavras_China))

todas_palavras_EUA = ' '.join([texto for texto in data_EUA["tratamento_5"]])
frequencia_EUA = nltk.FreqDist(token_espaco.tokenize(todas_palavras_EUA))

todas_palavras_Egito = ' '.join([texto for texto in data_Egito["tratamento_5"]])
frequencia_Egito = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Egito))

todas_palavras_Singapura = ' '.join([texto for texto in data_Singapura["tratamento_5"]])
frequencia_Singapura = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Singapura))

todas_palavras_UE = ' '.join([texto for texto in data_UE["tratamento_5"]])
frequencia_UE = nltk.FreqDist(token_espaco.tokenize(todas_palavras_UE))

todas_palavras_Vietna = ' '.join([texto for texto in data_Vietna["tratamento_5"]])
frequencia_Vietna = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Vietna))

todas_palavras_Ruanda = ' '.join([texto for texto in data_Ruanda["tratamento_5"]])
frequencia_Ruanda = nltk.FreqDist(token_espaco.tokenize(todas_palavras_Ruanda))

todas_palavras_ReinoUnido = ' '.join([texto for texto in data_ReinoUnido["tratamento_5"]])
frequencia_ReinoUnido = nltk.FreqDist(token_espaco.tokenize(todas_palavras_ReinoUnido))

Importando a biblioteca Pandas, para a exibição da tabela com os tokens mais citados

In [ ]:
import pandas as pd

In [ ]:
df_frequencia_Brasil = pd.DataFrame({"Brasil":list(frequencia_Brasil.keys()), "Frequência": list(frequencia_Brasil.values())})
lista_Brasil = df_frequencia_Brasil.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_China = pd.DataFrame({"China":list(frequencia_China.keys()), "Frequência": list(frequencia_China.values())})
lista_China = df_frequencia_China.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_EUA = pd.DataFrame({"EUA":list(frequencia_EUA.keys()), "Frequência": list(frequencia_EUA.values())})
lista_EUA = df_frequencia_EUA.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Egito = pd.DataFrame({"Egito":list(frequencia_Egito.keys()), "Frequência": list(frequencia_Egito.values())})
lista_Egito = df_frequencia_Egito.nlargest(columns = "Frequência", n = 12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Singapura = pd.DataFrame({"Singapura":list(frequencia_Singapura.keys()), "Frequência": list(frequencia_Singapura.values())})
lista_Singapura = df_frequencia_Singapura.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_UE = pd.DataFrame({"UE":list(frequencia_UE.keys()), "Frequência": list(frequencia_UE.values())})
lista_UE = df_frequencia_UE.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Vietna = pd.DataFrame({"Vietnã":list(frequencia_Vietna.keys()), "Frequência": list(frequencia_Vietna.values())})
lista_Vietna = df_frequencia_Vietna.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_Ruanda = pd.DataFrame({"Ruanda":list(frequencia_Ruanda.keys()), "Frequência": list(frequencia_Ruanda.values())})
lista_Ruanda = df_frequencia_Ruanda.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

In [ ]:
df_frequencia_ReinoUnido = pd.DataFrame({"Reino Unido":list(frequencia_ReinoUnido.keys()), "Frequência": list(frequencia_ReinoUnido.values())})
lista_ReinoUnido = df_frequencia_ReinoUnido.nlargest(columns = "Frequência", n =12) #Esse método mostra as primeiras palvras que mais aparecem

Gerando as tabelas lado a lado

In [ ]:
from IPython.display import HTML
def  side_by_side_country ( *list ):
  html = '<div style="display:grid; grid-template-columns: repeat(5,280px);">'
  for df in list:
      html += '<div style="margin-right: 2em">'
      html += df.to_html()
      html += '</div>'
  html += '</div>'
  display(HTML(html))

In [ ]:
side_by_side_country(lista_Brasil, lista_China, lista_Egito, lista_EUA, lista_ReinoUnido, lista_Ruanda,lista_Singapura, lista_UE, lista_Vietna)

,Brasil,Frequência
8,inteligência,175
9,artificial,174
47,dados,54
5,desenvolvimento,37
14,direitos,37
241,pessoas,35
553,risco,34
184,autoridade,29
57,público,28
508,impacto,25


#Referências

[Linguagem Natural parte 1: NLP com análise de sentimentok](https://www.alura.com.br/curso-online-introducao-a-nlp-com-analise-de-sentimento)

[Linguagem Natural parte 2: continuando com a análise de sentimento](https://www.alura.com.br/curso-online-nlp-com-analise-de-sentimento)

[NLP: regex e modelos de linguagem](https://www.alura.com.br/curso-online-nlp-modelos-linguagem)


[Expressões regularem em Python](https://www.acervolima.com.br/2021/05/expressoes-regulares-em-python.html)

[Displaying Multiple DataFrames Side By Side in Jupyter Lab/Notebook](https://python.plainenglish.io/displaying-multiple-dataframes-side-by-side-in-jupyter-lab-notebook-9a4649a4940)